## 1. Introduction

### 1.1 Contexte

L'apprentissage par renforcement (Reinforcement Learning, RL) est une branche du Machine Learning où un agent apprend à prendre des décisions en interagissant avec un environnement. Contrairement à l'apprentissage supervisé, l'agent ne dispose pas d'exemples étiquetés mais doit découvrir par essai-erreur quelles actions maximisent une récompense cumulative.

Le jeu Snake constitue un cas d'étude particulièrement intéressant pour le RL car il présente :
- Un espace d'états de taille variable (le serpent grandit)
- Des contraintes de collision (murs et corps du serpent)
- Un objectif clair (manger les pommes tout en survivant)
- Un compromis exploration/exploitation (chercher la pomme vs éviter les dangers)

### 1.2 Objectifs du projet

Ce projet vise à implémenter un agent capable de jouer au jeu Snake de manière autonome en utilisant l'algorithme PPO (Proximal Policy Optimization). Nous avons développé plusieurs versions de l'environnement et de l'entraînement pour étudier l'impact des différents choix de conception sur les performances de l'agent.

## 2. Fondements théoriques

### 2.1 Processus de Décision Markovien (MDP)

L'apprentissage par renforcement modélise le problème comme un **Processus de Décision Markovien** défini par le tuple $(S, A, P, R, \gamma)$ :

- $S$ : Ensemble des états possibles
- $A$ : Ensemble des actions possibles
- $P(s'|s,a)$ : Probabilité de transition vers l'état $s'$ depuis $s$ en effectuant $a$
- $R(s,a,s')$ : Récompense obtenue lors de la transition
- $\gamma \in [0,1]$ : Facteur d'actualisation (discount factor)

L'objectif est de trouver une **politique** $\pi(a|s)$ qui maximise l'espérance du retour cumulé :

$$G_t = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$$

### 2.2 L'algorithme PPO (Proximal Policy Optimization)

PPO est un algorithme de type **Policy Gradient** développé par OpenAI en 2017. Il améliore la stabilité de l'apprentissage en limitant les mises à jour trop importantes de la politique.

La fonction objectif de PPO utilise un **clipping** pour contraindre le ratio entre l'ancienne et la nouvelle politique :

$$L^{CLIP}(\theta) = \mathbb{E}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t \right) \right]$$

Où :
- $r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$ est le ratio de probabilité
- $\hat{A}_t$ est l'estimateur de l'avantage
- $\epsilon$ est le paramètre de clipping (typiquement 0.2)

### 2.3 Architecture Actor-Critic

PPO utilise une architecture **Actor-Critic** composée de deux réseaux :

- **Actor** (Politique) : Produit une distribution de probabilité sur les actions
- **Critic** (Valeur) : Estime la valeur d'un état $V(s)$

Dans notre cas, nous utilisons un réseau MLP (Multi-Layer Perceptron) partagé avec deux têtes de sortie.

## 3. Implémentation de l'environnement Snake

L'environnement est le coeur du projet. Il définit les règles du jeu, l'espace d'observation que l'agent perçoit, et le système de récompenses qui guide l'apprentissage.

### 3.1 Spécifications de l'environnement

| Paramètre | Valeur | Description |
|-----------|--------|-------------|
| Taille de la grille | 30x30 cases | Fenêtre de 600x600 pixels avec blocs de 20px |
| Espace d'actions | Discrete(4) | Gauche, Droite, Haut, Bas |
| Espace d'observations | Box(11,) | Vecteur de 11 valeurs binaires |
| Récompense positive | +10 | Manger une pomme |
| Récompense négative | -10 | Collision (mur ou corps) |

### 3.2 Configuration initiale et imports

In [1]:
# Configuration pour éviter les problèmes de threads sur macOS
import os
os.environ["OMP_NUM_THREADS"] = "1"

import torch
torch.set_num_threads(1)

# Imports principaux
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random
import pygame

print(f"Gymnasium version: {gym.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")

Gymnasium version: 1.2.3
PyTorch version: 2.9.1
NumPy version: 2.2.6


/Users/samye/.pyenv/versions/3.13.0/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


### 3.3 Constantes du jeu

Nous définissons les paramètres de base du jeu : dimensions de la fenêtre, taille des blocs, et palette de couleurs pour le rendu graphique.

In [2]:
# Dimensions du jeu
WINDOW_WIDTH = 600
WINDOW_HEIGHT = 600
BLOCK_SIZE = 20  # Taille d'une case en pixels
SPEED = 20       # FPS pour le rendu humain

# Palette de couleurs
WHITE = (255, 255, 255)
BLACK = (15, 15, 25)
DARK_GRAY = (30, 30, 40)
RED = (255, 80, 80)
ORANGE = (255, 165, 0)
GREEN = (76, 175, 80)
BLUE1 = (66, 165, 245)
CYAN = (0, 188, 212)
YELLOW = (255, 235, 59)

# Calcul de la taille de la grille
GRID_WIDTH = WINDOW_WIDTH // BLOCK_SIZE
GRID_HEIGHT = WINDOW_HEIGHT // BLOCK_SIZE
print(f"Grille de jeu : {GRID_WIDTH}x{GRID_HEIGHT} cases")
print(f"Nombre total de positions possibles : {GRID_WIDTH * GRID_HEIGHT}")

Grille de jeu : 30x30 cases
Nombre total de positions possibles : 900


### 3.4 Classe SnakeEnv : Structure principale

L'environnement hérite de `gym.Env` et doit implémenter les méthodes standard :
- `__init__()` : Initialisation des espaces d'action et d'observation
- `reset()` : Réinitialisation de l'environnement
- `step(action)` : Exécution d'une action et retour de l'observation, récompense, etc.

In [3]:
class SnakeEnv(gym.Env):
    """
    Environnement Snake compatible avec Gymnasium et Stable-Baselines3.
    
    L'agent contrôle un serpent qui doit manger des pommes pour grandir
    tout en évitant les collisions avec les murs et son propre corps.
    """
    metadata = {'render_modes': ['human'], 'render_fps': SPEED}

    def __init__(self, render_mode=None):
        super(SnakeEnv, self).__init__()
        
        # Dimensions de la grille
        self.w = WINDOW_WIDTH
        self.h = WINDOW_HEIGHT
        self.render_mode = render_mode
        
        # Variables Pygame (initialisées à la demande)
        self.window = None
        self.clock = None
        self.font = None
        self.small_font = None
        
        # ESPACE D'ACTIONS : 4 directions possibles
        # 0: Gauche, 1: Droite, 2: Haut, 3: Bas
        self.action_space = spaces.Discrete(4)
        
        # ESPACE D'OBSERVATIONS : Vecteur de 11 valeurs binaires
        # [danger_devant, danger_droite, danger_gauche,
        #  dir_gauche, dir_droite, dir_haut, dir_bas,
        #  pomme_gauche, pomme_droite, pomme_haut, pomme_bas]
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(11,), dtype=np.int8
        )
    
    def reset(self, seed=None, options=None):
        """Réinitialise l'environnement pour un nouvel épisode."""
        super().reset(seed=seed)
        
        # Position initiale : serpent au centre, orienté vers la droite
        self.direction = 1  # 0:G, 1:D, 2:H, 3:B
        self.head = [self.w / 2, self.h / 2]
        self.snake = [
            self.head,
            [self.head[0] - BLOCK_SIZE, self.head[1]],
            [self.head[0] - 2 * BLOCK_SIZE, self.head[1]]
        ]
        
        self.score = 0
        self.frame_iteration = 0
        self._place_food()
        
        return self._get_observation(), {}
    
    def step(self, action):
        """Exécute une action et retourne le nouvel état."""
        self.frame_iteration += 1
        
        # 1. Déplacer le serpent
        self._move(action)
        
        # 2. Vérifier les collisions
        game_over = False
        reward = 0
        
        # Condition de fin : collision ou trop de frames sans manger
        if self._is_collision() or self.frame_iteration > 100 * len(self.snake):
            game_over = True
            reward = -10
            return self._get_observation(), reward, game_over, False, {}
        
        # 3. Vérifier si le serpent mange la pomme
        if self.head == self.food:
            self.score += 1
            reward = 10
            self._place_food()
        else:
            # Pas de pomme mangée : retirer la queue (mouvement normal)
            self.snake.pop()
        
        # 4. Rendu visuel si demandé
        if self.render_mode == "human":
            self._render_frame()
        
        return self._get_observation(), reward, game_over, False, {}
    
    def _get_observation(self):
        """
        Construit le vecteur d'observation de 11 valeurs.
        
        C'est la représentation de l'état que l'agent perçoit.
        On encode les informations de manière relative à la direction actuelle.
        """
        head = self.snake[0]
        
        # Points adjacents à la tête
        point_l = [head[0] - BLOCK_SIZE, head[1]]
        point_r = [head[0] + BLOCK_SIZE, head[1]]
        point_u = [head[0], head[1] - BLOCK_SIZE]
        point_d = [head[0], head[1] + BLOCK_SIZE]
        
        # Direction actuelle (booléens)
        dir_l = self.direction == 0
        dir_r = self.direction == 1
        dir_u = self.direction == 2
        dir_d = self.direction == 3
        
        state = [
            # Danger tout droit (relatif à la direction)
            (dir_r and self._is_collision(point_r)) or
            (dir_l and self._is_collision(point_l)) or
            (dir_u and self._is_collision(point_u)) or
            (dir_d and self._is_collision(point_d)),
            
            # Danger à droite (relatif)
            (dir_u and self._is_collision(point_r)) or
            (dir_d and self._is_collision(point_l)) or
            (dir_l and self._is_collision(point_u)) or
            (dir_r and self._is_collision(point_d)),
            
            # Danger à gauche (relatif)
            (dir_d and self._is_collision(point_r)) or
            (dir_u and self._is_collision(point_l)) or
            (dir_r and self._is_collision(point_u)) or
            (dir_l and self._is_collision(point_d)),
            
            # Direction actuelle (one-hot encoding)
            dir_l,
            dir_r,
            dir_u,
            dir_d,
            
            # Position relative de la pomme
            self.food[0] < self.head[0],  # Pomme à gauche
            self.food[0] > self.head[0],  # Pomme à droite
            self.food[1] < self.head[1],  # Pomme en haut
            self.food[1] > self.head[1]   # Pomme en bas
        ]
        
        return np.array(state, dtype=np.int8)
    
    def _place_food(self):
        """Place la pomme à une position aléatoire libre."""
        x = random.randint(0, (self.w - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
        y = random.randint(0, (self.h - BLOCK_SIZE) // BLOCK_SIZE) * BLOCK_SIZE
        self.food = [x, y]
        
        # S'assurer que la pomme n'est pas sur le serpent
        if self.food in self.snake:
            self._place_food()
    
    def _is_collision(self, pt=None):
        """Vérifie si un point est en collision avec un mur ou le corps."""
        if pt is None:
            pt = self.head
        
        # Collision avec les murs
        if pt[0] > self.w - BLOCK_SIZE or pt[0] < 0:
            return True
        if pt[1] > self.h - BLOCK_SIZE or pt[1] < 0:
            return True
        
        # Collision avec le corps (sauf la tête)
        if pt in self.snake[1:]:
            return True
        
        return False
    
    def _move(self, action):
        """Déplace le serpent dans la direction demandée."""
        # Empêcher le demi-tour (collision immédiate avec soi-même)
        if action == 0 and self.direction != 1:
            self.direction = 0  # Gauche
        elif action == 1 and self.direction != 0:
            self.direction = 1  # Droite
        elif action == 2 and self.direction != 3:
            self.direction = 2  # Haut
        elif action == 3 and self.direction != 2:
            self.direction = 3  # Bas
        
        # Calculer la nouvelle position de la tête
        x, y = self.head
        if self.direction == 0:
            x -= BLOCK_SIZE
        elif self.direction == 1:
            x += BLOCK_SIZE
        elif self.direction == 2:
            y -= BLOCK_SIZE
        elif self.direction == 3:
            y += BLOCK_SIZE
        
        self.head = [x, y]
        self.snake.insert(0, self.head)
    
    def _render_frame(self):
        """Affiche le jeu avec Pygame."""
        if self.window is None:
            pygame.init()
            self.window = pygame.display.set_mode((self.w, self.h))
            self.clock = pygame.time.Clock()
            self.font = pygame.font.Font(None, 48)
            self.small_font = pygame.font.Font(None, 32)
            pygame.display.set_caption("Snake AI")
        
        pygame.event.pump()
        self.window.fill(BLACK)
        
        # Grille de fond
        for x in range(0, self.w, BLOCK_SIZE):
            pygame.draw.line(self.window, DARK_GRAY, (x, 0), (x, self.h), 1)
        for y in range(0, self.h, BLOCK_SIZE):
            pygame.draw.line(self.window, DARK_GRAY, (0, y), (self.w, y), 1)
        
        # Pomme
        pygame.draw.rect(
            self.window, RED,
            pygame.Rect(self.food[0], self.food[1], BLOCK_SIZE, BLOCK_SIZE)
        )
        
        # Serpent
        for i, pt in enumerate(self.snake):
            color = CYAN if i == 0 else BLUE1
            pygame.draw.rect(
                self.window, color,
                pygame.Rect(pt[0], pt[1], BLOCK_SIZE - 1, BLOCK_SIZE - 1)
            )
        
        # Score
        text = self.font.render(f"Score: {self.score}", True, GREEN)
        self.window.blit(text, (10, 10))
        
        pygame.display.flip()
        self.clock.tick(self.metadata["render_fps"])
    
    def close(self):
        """Ferme proprement l'environnement."""
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

### 3.5 Explication du vecteur d'observation

Le choix d'un vecteur de 11 valeurs binaires est crucial. Cette représentation compacte encode toutes les informations nécessaires à la prise de décision :

| Index | Description | Utilité |
|-------|-------------|--------|
| 0 | Danger tout droit | Éviter la collision frontale |
| 1 | Danger à droite | Éviter la collision latérale |
| 2 | Danger à gauche | Éviter la collision latérale |
| 3-6 | Direction actuelle (one-hot) | Contexte de mouvement |
| 7-10 | Position relative de la pomme | Objectif à atteindre |

Cette représentation est **relative** à la direction du serpent, ce qui simplifie l'apprentissage car l'agent n'a pas besoin d'apprendre les symétries du jeu.

### 3.6 Test de l'environnement

Vérifions que l'environnement fonctionne correctement avec quelques actions aléatoires.

In [4]:
# Test de l'environnement
env = SnakeEnv()

# Vérification des espaces
print("Espace d'actions:", env.action_space)
print("Espace d'observations:", env.observation_space)
print()

# Test d'un reset
obs, info = env.reset(seed=42)
print("Observation initiale:")
print(f"  Forme: {obs.shape}")
print(f"  Valeurs: {obs}")
print(f"  Position tête: {env.head}")
print(f"  Position pomme: {env.food}")
print()

# Test de quelques steps
print("Test de 5 actions aléatoires:")
for i in range(5):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    action_names = ['Gauche', 'Droite', 'Haut', 'Bas']
    print(f"  Action {i+1}: {action_names[action]}, Reward: {reward}, Done: {done}")
    if done:
        break

env.close()

Espace d'actions: Discrete(4)
Espace d'observations: Box(0, 1, (11,), int8)

Observation initiale:
  Forme: (11,)
  Valeurs: [0 0 0 0 1 0 0 1 0 0 1]
  Position tête: [300.0, 300.0]
  Position pomme: [80, 420]

Test de 5 actions aléatoires:
  Action 1: Haut, Reward: 0, Done: False
  Action 2: Bas, Reward: 0, Done: False
  Action 3: Haut, Reward: 0, Done: False
  Action 4: Gauche, Reward: 0, Done: False
  Action 5: Haut, Reward: 0, Done: False


### 3.7 Validation avec Stable-Baselines3

Stable-Baselines3 fournit un outil pour vérifier la compatibilité de l'environnement.

In [5]:
from stable_baselines3.common.env_checker import check_env

# Vérification de compatibilité
env = SnakeEnv()
try:
    check_env(env, warn=True)
    print("L'environnement est compatible avec Stable-Baselines3.")
except Exception as e:
    print(f"Erreur de compatibilité: {e}")
finally:
    env.close()

L'environnement est compatible avec Stable-Baselines3.


---

## 4. Version 1 : Premier entraînement

Cette première version constitue notre baseline. L'objectif est d'obtenir un agent fonctionnel avec une configuration minimale pour établir une référence de performance.

### 4.1 Configuration de la V1

| Paramètre | Valeur | Justification |
|-----------|--------|---------------|
| Algorithme | PPO | Stable et efficace pour les espaces discrets |
| Politique | MlpPolicy | Réseau fully-connected adapté aux observations vectorielles |
| Architecture | [64, 64] | Configuration par défaut de SB3 |
| Timesteps | 100 000 | Premier test rapide |
| Learning rate | 0.0003 | Valeur par défaut |

### 4.2 Code d'entraînement V1

In [6]:
from stable_baselines3 import PPO

# Création des dossiers de sauvegarde
MODELS_DIR_V1 = "checkpoints/PPO_v1"
LOG_DIR = "logs"

os.makedirs(MODELS_DIR_V1, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# Création de l'environnement (sans rendu pour la vitesse)
env_v1 = SnakeEnv()

# Initialisation du modèle PPO
# MlpPolicy : réseau de neurones standard [64, 64] par défaut
model_v1 = PPO(
    policy="MlpPolicy",
    env=env_v1,
    verbose=1,
    tensorboard_log=LOG_DIR
)

print("Modèle V1 initialisé")
print(f"Architecture du réseau : {model_v1.policy}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Modèle V1 initialisé
Architecture du réseau : ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=11, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=11, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=4, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=

### 4.3 Lancement de l'entraînement V1

L'entraînement de 100 000 timesteps prend environ 2-3 minutes sur un processeur moderne.

In [7]:
# Nombre de pas d'entraînement
TIMESTEPS_V1 = 100_000

print(f"Démarrage de l'entraînement V1 ({TIMESTEPS_V1:,} timesteps)...")
print("-" * 50)

# Entraînement
model_v1.learn(total_timesteps=TIMESTEPS_V1)

# Sauvegarde du modèle
model_v1.save(f"{MODELS_DIR_V1}/snake_v1_final")
print("-" * 50)
print(f"Entraînement V1 terminé !")
print(f"Modèle sauvegardé : {MODELS_DIR_V1}/snake_v1_final.zip")

Démarrage de l'entraînement V1 (100,000 timesteps)...
--------------------------------------------------
Logging to logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 118      |
|    ep_rew_mean     | -9.41    |
| time/              |          |
|    fps             | 7111     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 118      |
|    ep_rew_mean     | -9.41    |
| time/              |          |
|    fps             | 7111     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 98.8       |
|    ep_rew_mean          | -9.02      |
| time/                   |  

### 4.4 Évaluation de la V1

Testons les performances du modèle entraîné sur plusieurs épisodes.

In [8]:
def evaluate_model(model, env, n_episodes=10):
    """
    Évalue un modèle sur plusieurs épisodes.
    
    Returns:
        scores: liste des scores obtenus
        lengths: liste des longueurs d'épisode
    """
    scores = []
    lengths = []
    
    for episode in range(n_episodes):
        obs, _ = env.reset()
        done = False
        episode_length = 0
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            episode_length += 1
        
        scores.append(env.score)
        lengths.append(episode_length)
    
    return scores, lengths

# Évaluation sur 20 épisodes
env_eval = SnakeEnv()
scores_v1, lengths_v1 = evaluate_model(model_v1, env_eval, n_episodes=20)
env_eval.close()

print("Résultats V1 sur 20 épisodes:")
print(f"  Score moyen     : {np.mean(scores_v1):.2f} (+/- {np.std(scores_v1):.2f})")
print(f"  Score max       : {max(scores_v1)}")
print(f"  Score min       : {min(scores_v1)}")
print(f"  Durée moyenne   : {np.mean(lengths_v1):.0f} steps")

Résultats V1 sur 20 épisodes:
  Score moyen     : 19.65 (+/- 8.24)
  Score max       : 40
  Score min       : 1
  Durée moyenne   : 411 steps


### 4.5 Visualisation du jeu V1

Lancez cette cellule pour voir l'agent V1 jouer en temps réel. Fermez la fenêtre Pygame pour arrêter.

In [2]:
import time

def play_game(model, n_games=3, delay=0.05):
    """
    Lance le jeu avec rendu graphique pour visualiser l'agent.
    
    Args:
        model: Le modèle PPO entraîné
        n_games: Nombre de parties à jouer
        delay: Délai entre chaque frame (en secondes)
    """
    env_play = SnakeEnv(render_mode="human")
    
    for game in range(n_games):
        obs, _ = env_play.reset()
        done = False
        print(f"\nPartie {game + 1}/{n_games}")
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env_play.step(action)
            time.sleep(delay)
        
        print(f"  Score final : {env_play.score}")
    
    env_play.close()
    print("\nVisualisations terminées.")

# Lancer 3 parties avec le modèle V1
print("Lancement du jeu avec le modèle V1...")
play_game(model_v1, n_games=3, delay=0.05)

Lancement du jeu avec le modèle V1...


NameError: name 'model_v1' is not defined

### 4.5 Analyse de la V1

La version 1 présente plusieurs limitations :

1. **Nombre de timesteps insuffisant** : 100k steps ne permettent pas une convergence optimale
2. **Architecture réseau limitée** : Le réseau [64, 64] par défaut peut manquer de capacité
3. **Pas de checkpoints** : Impossible de reprendre l'entraînement ou d'analyser la progression

Ces observations motivent le développement de la version 2.

---

## 5. Version 2 : Entraînement avancé

La version 2 améliore significativement l'entraînement avec :
- Un réseau de neurones plus large
- Plus de timesteps d'entraînement
- Un système de checkpoints pour sauvegarder la progression

### 5.1 Configuration de la V2

| Paramètre | V1 | V2 | Justification |
|-----------|----|----|---------------|
| Architecture | [64, 64] | [128, 128] | Plus de capacité d'apprentissage |
| Timesteps | 100 000 | 2 000 000 | Convergence plus complète |
| Learning rate | 0.0003 | 0.0003 | Valeur standard efficace |
| Gamma | 0.99 | 0.99 | Vision long terme |
| Checkpoints | Non | Tous les 100k | Suivi de progression |

### 5.2 Code d'entraînement V2

In [9]:
from stable_baselines3.common.callbacks import CheckpointCallback

# Configuration V2
MODELS_DIR_V2 = "checkpoints/PPO_v2"
TIMESTEPS_V2 = 2_000_000  # 2 millions de pas
SAVE_FREQ = 100_000       # Sauvegarde tous les 100k pas

os.makedirs(MODELS_DIR_V2, exist_ok=True)

# Création de l'environnement
env_v2 = SnakeEnv()

# Callback pour sauvegarder des checkpoints réguliers
checkpoint_callback = CheckpointCallback(
    save_freq=SAVE_FREQ,
    save_path=MODELS_DIR_V2,
    name_prefix="snake_v2"
)

# Architecture personnalisée : réseau plus large
policy_kwargs = dict(
    net_arch=[128, 128]  # Deux couches de 128 neurones
)

# Initialisation du modèle V2
model_v2 = PPO(
    policy="MlpPolicy",
    env=env_v2,
    verbose=1,
    tensorboard_log=LOG_DIR,
    learning_rate=0.0003,
    gamma=0.99,
    policy_kwargs=policy_kwargs
)

print("Modèle V2 initialisé")
print(f"Architecture : {policy_kwargs['net_arch']}")
print(f"Timesteps prévus : {TIMESTEPS_V2:,}")
print(f"Checkpoints tous les {SAVE_FREQ:,} steps")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Modèle V2 initialisé
Architecture : [128, 128]
Timesteps prévus : 2,000,000
Checkpoints tous les 100,000 steps


### 5.3 Lancement de l'entraînement V2

L'entraînement de 2 millions de timesteps prend environ 20-30 minutes.

In [10]:
print(f"Démarrage de l'entraînement V2 ({TIMESTEPS_V2:,} timesteps)...")
print(f"Checkpoints sauvegardés dans : {MODELS_DIR_V2}")
print("-" * 50)

# Entraînement avec callback
model_v2.learn(
    total_timesteps=TIMESTEPS_V2,
    callback=checkpoint_callback
)

print("-" * 50)
print("Entraînement V2 terminé !")

Démarrage de l'entraînement V2 (2,000,000 timesteps)...
Checkpoints sauvegardés dans : checkpoints/PPO_v2
--------------------------------------------------
Logging to logs/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.8     |
|    ep_rew_mean     | -9.55    |
| time/              |          |
|    fps             | 7069     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.8     |
|    ep_rew_mean     | -9.55    |
| time/              |          |
|    fps             | 7069     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 102         |
|    ep_rew_mea

### 5.4 Évaluation de la V2

In [11]:
# Évaluation sur 20 épisodes
env_eval_v2 = SnakeEnv()
scores_v2, lengths_v2 = evaluate_model(model_v2, env_eval_v2, n_episodes=20)
env_eval_v2.close()

print("Résultats V2 sur 20 épisodes:")
print(f"  Score moyen     : {np.mean(scores_v2):.2f} (+/- {np.std(scores_v2):.2f})")
print(f"  Score max       : {max(scores_v2)}")
print(f"  Score min       : {min(scores_v2)}")
print(f"  Durée moyenne   : {np.mean(lengths_v2):.0f} steps")

Résultats V2 sur 20 épisodes:
  Score moyen     : 25.50 (+/- 11.69)
  Score max       : 52
  Score min       : 11
  Durée moyenne   : 534 steps


### 5.5 Visualisation du jeu V2

Lancez cette cellule pour voir l'agent V2 jouer en temps réel. Fermez la fenêtre Pygame pour arrêter.

In [15]:
# Lancer 3 parties avec le modèle V2
print("Lancement du jeu avec le modèle V2...")
play_game(model_v2, n_games=3, delay=0.05)

Lancement du jeu avec le modèle V2...

Partie 1/3
  Score final : 27

Partie 2/3
  Score final : 27

Partie 2/3
  Score final : 24

Partie 3/3
  Score final : 24

Partie 3/3
  Score final : 25

Visualisations terminées.
  Score final : 25

Visualisations terminées.


---

## 6. Version 3 : Vision par Convolution (CNN)

### 6.1 Analyse des limitations de la V2

Les versions V1 et V2 souffrent d'un problème fondamental : **la myopie**. L'agent ne perçoit que :
- Les dangers immédiats (cases adjacentes à la tête)
- La direction générale de la pomme

**Ce que l'agent ne voit pas :**
- La forme complète de son propre corps
- Les "culs-de-sac" qu'il crée en se déplaçant
- Les impasses formées par sa queue

**Conséquence :** Dès que le serpent devient long, il finit par s'enfermer lui-même car il ne peut pas anticiper qu'il entre dans une zone fermée.

### 6.2 La solution : Représentation spatiale

Pour résoudre ce problème, nous passons d'une représentation vectorielle (11 valeurs) à une **représentation matricielle** de la grille complète. L'agent "voit" le plateau comme une image.

| Aspect | V1/V2 (Vecteur) | V3 (Image) |
|--------|-----------------|------------|
| Observation | Box(11,) | Box(30, 30, 3) |
| Information | Locale (dangers proches) | Globale (grille entière) |
| Réseau | MLP [64, 64] ou [128, 128] | CNN (Convolutions) |
| Capacité | Réactive | Anticipative |

### 6.3 Codage de l'observation

La grille est encodée comme une "image" RGB où chaque canal représente un élément :

| Canal | Élément | Valeur |
|-------|---------|--------|
| 0 (Rouge) | Corps du serpent | 255 si présent, 0 sinon |
| 1 (Vert) | Tête du serpent | 255 si présent, 0 sinon |
| 2 (Bleu) | Pomme | 255 si présent, 0 sinon |

Cette représentation permet au CNN de détecter les patterns spatiaux (formes en U, impasses, etc.).

### 6.4 Implémentation de l'environnement CNN

L'environnement `SnakeEnvCNN` modifie principalement la méthode `_get_observation()` pour retourner une matrice 3D représentant la grille complète.

In [ ]:
class SnakeEnvCNN(gym.Env):
    """
    Environnement Snake avec observation par image (CNN).
    
    L'agent perçoit la grille complète comme une matrice 3D,
    permettant aux réseaux convolutifs de détecter les patterns spatiaux.
    
    Reward Shaping anti-oscillation :
    - +10   : Manger une pomme
    - -10   : Collision (game over)
    - +0.1  : Se rapprocher de la pomme
    - -0.3  : S'éloigner de la pomme (asymétrique pour éviter l'oscillation)
    - -0.01 : Coût par step (encourage l'efficacité)
    """
    metadata = {'render_modes': ['human'], 'render_fps': SPEED}

    def __init__(self, render_mode=None):
        super(SnakeEnvCNN, self).__init__()
        
        # Dimensions de la grille en cases (pas en pixels)
        self.grid_w = GRID_WIDTH   # 30 cases
        self.grid_h = GRID_HEIGHT  # 30 cases
        self.w = WINDOW_WIDTH
        self.h = WINDOW_HEIGHT
        self.render_mode = render_mode
        
        # Variables Pygame
        self.window = None
        self.clock = None
        self.font = None
        
        # ESPACE D'ACTIONS : 4 directions possibles
        self.action_space = spaces.Discrete(4)
        
        # ESPACE D'OBSERVATIONS : Image de la grille
        # Shape: (hauteur, largeur, canaux) = (30, 30, 3)
        # Canal 0: Corps du serpent
        # Canal 1: Tête du serpent  
        # Canal 2: Pomme
        self.observation_space = spaces.Box(
            low=0, 
            high=255, 
            shape=(self.grid_h, self.grid_w, 3), 
            dtype=np.uint8
        )
    
    def _get_distance_to_food(self):
        """Calcule la distance de Manhattan entre la tête et la pomme."""
        return abs(self.head[0] - self.food[0]) + abs(self.head[1] - self.food[1])
    
    def reset(self, seed=None, options=None):
        """Réinitialise l'environnement."""
        super().reset(seed=seed)
        
        # Position initiale au centre de la grille (en coordonnées de grille)
        center_x = self.grid_w // 2
        center_y = self.grid_h // 2
        
        self.direction = 1  # 0:G, 1:D, 2:H, 3:B
        self.head = [center_x, center_y]
        self.snake = [
            self.head.copy(),
            [center_x - 1, center_y],
            [center_x - 2, center_y]
        ]
        
        self.score = 0
        self.frame_iteration = 0
        self._place_food()
        
        # Sauvegarder la distance initiale pour le reward shaping
        self.prev_distance = self._get_distance_to_food()
        
        return self._get_observation(), {}
    
    def step(self, action):
        """Exécute une action avec reward shaping anti-oscillation."""
        self.frame_iteration += 1
        
        # Déplacer le serpent
        self._move(action)
        
        # Vérifier les collisions
        game_over = False
        reward = -0.01  # Coût de base par step (encourage l'efficacité)
        
        # GAME OVER : Collision ou timeout
        if self._is_collision() or self.frame_iteration > 100 * len(self.snake):
            game_over = True
            reward = -10
            return self._get_observation(), reward, game_over, False, {}
        
        # Calculer la nouvelle distance à la pomme
        current_distance = self._get_distance_to_food()
        
        # MANGER LA POMME : Récompense principale
        if self.head == self.food:
            self.score += 1
            reward = 10
            self._place_food()
            # Réinitialiser la distance de référence après avoir mangé
            self.prev_distance = self._get_distance_to_food()
        else:
            # REWARD SHAPING ASYMÉTRIQUE : Empêche l'oscillation
            if current_distance < self.prev_distance:
                reward += 0.1   # Se rapproche : petite récompense
            elif current_distance > self.prev_distance:
                reward += -0.3  # S'éloigne : pénalité plus forte
            # Si même distance : juste le coût de base (-0.01)
            
            self.prev_distance = current_distance
            self.snake.pop()
        
        if self.render_mode == "human":
            self._render_frame()
        
        return self._get_observation(), reward, game_over, False, {}
    
    def _get_observation(self):
        """
        Construit l'observation comme une image de la grille.
        
        Returns:
            np.array de shape (30, 30, 3) en uint8
            - Canal 0: Corps du serpent (255 si présent)
            - Canal 1: Tête du serpent (255 si présent)
            - Canal 2: Pomme (255 si présent)
        """
        # Initialiser la grille vide
        obs = np.zeros((self.grid_h, self.grid_w, 3), dtype=np.uint8)
        
        # Canal 0: Corps du serpent (sans la tête)
        for segment in self.snake[1:]:
            x, y = int(segment[0]), int(segment[1])
            if 0 <= x < self.grid_w and 0 <= y < self.grid_h:
                obs[y, x, 0] = 255
        
        # Canal 1: Tête du serpent
        hx, hy = int(self.head[0]), int(self.head[1])
        if 0 <= hx < self.grid_w and 0 <= hy < self.grid_h:
            obs[hy, hx, 1] = 255
        
        # Canal 2: Pomme
        fx, fy = int(self.food[0]), int(self.food[1])
        if 0 <= fx < self.grid_w and 0 <= fy < self.grid_h:
            obs[fy, fx, 2] = 255
        
        return obs
    
    def _place_food(self):
        """Place la pomme à une position aléatoire libre (en coordonnées de grille)."""
        x = random.randint(0, self.grid_w - 1)
        y = random.randint(0, self.grid_h - 1)
        self.food = [x, y]
        
        if self.food in self.snake:
            self._place_food()
    
    def _is_collision(self, pt=None):
        """Vérifie les collisions (en coordonnées de grille)."""
        if pt is None:
            pt = self.head
        
        # Collision avec les murs
        if pt[0] >= self.grid_w or pt[0] < 0:
            return True
        if pt[1] >= self.grid_h or pt[1] < 0:
            return True
        
        # Collision avec le corps
        if pt in self.snake[1:]:
            return True
        
        return False
    
    def _move(self, action):
        """Déplace le serpent (en coordonnées de grille)."""
        # Empêcher le demi-tour
        if action == 0 and self.direction != 1:
            self.direction = 0
        elif action == 1 and self.direction != 0:
            self.direction = 1
        elif action == 2 and self.direction != 3:
            self.direction = 2
        elif action == 3 and self.direction != 2:
            self.direction = 3
        
        x, y = self.head
        if self.direction == 0:
            x -= 1
        elif self.direction == 1:
            x += 1
        elif self.direction == 2:
            y -= 1
        elif self.direction == 3:
            y += 1
        
        self.head = [x, y]
        self.snake.insert(0, self.head.copy())
    
    def _render_frame(self):
        """Affiche le jeu avec Pygame."""
        if self.window is None:
            pygame.init()
            self.window = pygame.display.set_mode((self.w, self.h))
            self.clock = pygame.time.Clock()
            self.font = pygame.font.Font(None, 48)
            pygame.display.set_caption("Snake AI - CNN")
        
        pygame.event.pump()
        self.window.fill(BLACK)
        
        # Grille
        for x in range(0, self.w, BLOCK_SIZE):
            pygame.draw.line(self.window, DARK_GRAY, (x, 0), (x, self.h), 1)
        for y in range(0, self.h, BLOCK_SIZE):
            pygame.draw.line(self.window, DARK_GRAY, (0, y), (self.w, y), 1)
        
        # Pomme (convertir coordonnées grille -> pixels)
        pygame.draw.rect(
            self.window, RED,
            pygame.Rect(self.food[0] * BLOCK_SIZE, self.food[1] * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE)
        )
        
        # Serpent
        for i, pt in enumerate(self.snake):
            color = CYAN if i == 0 else BLUE1
            pygame.draw.rect(
                self.window, color,
                pygame.Rect(pt[0] * BLOCK_SIZE, pt[1] * BLOCK_SIZE, BLOCK_SIZE - 1, BLOCK_SIZE - 1)
            )
        
        # Score
        text = self.font.render(f"Score: {self.score}", True, GREEN)
        self.window.blit(text, (10, 10))
        
        pygame.display.flip()
        self.clock.tick(self.metadata["render_fps"])
    
    def close(self):
        """Ferme l'environnement."""
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

print("Classe SnakeEnvCNN définie avec reward shaping anti-oscillation.")
print()
print("Système de récompenses V3 (anti-farming):")
print("  +10   : Manger une pomme")
print("  -10   : Collision (game over)")
print("  +0.1  : Se rapprocher de la pomme")
print("  -0.3  : S'éloigner de la pomme (3x plus pénalisant)")
print("  -0.01 : Coût par step (encourage l'efficacité)")
print()
print("→ Oscillation impossible : s'éloigner puis se rapprocher = -0.3 + 0.1 = -0.2")

### 6.5 Visualisation de l'observation CNN

Vérifions visuellement ce que l'agent perçoit avec cette nouvelle représentation.

In [ ]:
import matplotlib.pyplot as plt

# Test de l'environnement CNN
env_cnn_test = SnakeEnvCNN()
obs, _ = env_cnn_test.reset(seed=42)

# Faire quelques pas pour avoir un serpent plus long
for _ in range(10):
    action = env_cnn_test.action_space.sample()
    obs, _, done, _, _ = env_cnn_test.step(action)
    if done:
        obs, _ = env_cnn_test.reset()

print(f"Shape de l'observation : {obs.shape}")
print(f"Type : {obs.dtype}")
print(f"Valeurs min/max : {obs.min()}, {obs.max()}")

# Visualisation des 3 canaux
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Canal 0: Corps
axes[0].imshow(obs[:, :, 0], cmap='Reds')
axes[0].set_title('Canal 0: Corps du serpent')
axes[0].axis('off')

# Canal 1: Tête
axes[1].imshow(obs[:, :, 1], cmap='Greens')
axes[1].set_title('Canal 1: Tête du serpent')
axes[1].axis('off')

# Canal 2: Pomme
axes[2].imshow(obs[:, :, 2], cmap='Blues')
axes[2].set_title('Canal 2: Pomme')
axes[2].axis('off')

# Image RGB combinée
axes[3].imshow(obs)
axes[3].set_title('Image RGB combinée')
axes[3].axis('off')

plt.suptitle("Visualisation de l'observation CNN", fontsize=14)
plt.tight_layout()
plt.show()

env_cnn_test.close()

### 6.6 Validation de l'environnement CNN

In [ ]:
# Vérification de compatibilité avec Stable-Baselines3
env_cnn_check = SnakeEnvCNN()
try:
    check_env(env_cnn_check, warn=True)
    print("✓ L'environnement CNN est compatible avec Stable-Baselines3.")
except Exception as e:
    print(f"✗ Erreur de compatibilité: {e}")
finally:
    env_cnn_check.close()

### 6.7 Configuration de l'entraînement V3

La version CNN nécessite des ajustements par rapport aux versions MLP :

| Paramètre | V2 (MLP) | V3 (CNN) | Justification |
|-----------|----------|----------|---------------|
| Policy | MlpPolicy | CnnPolicy | Adapté aux entrées images |
| Architecture | [128, 128] | NatureCNN modifié | Convolutions pour patterns spatiaux |
| Timesteps | 2M | 3M+ | CNN plus lent à converger |
| Learning rate | 0.0003 | 0.0001 | Plus stable pour CNN |
| n_steps | 2048 | 2048 | Collecte plus d'expérience |

**Note sur l'architecture CNN** : Stable-Baselines3 utilise par défaut l'architecture "NatureCNN" (inspirée des travaux de DeepMind sur Atari). Nous allons définir une architecture personnalisée adaptée à notre grille 30x30.

In [ ]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn

class CustomCNN(BaseFeaturesExtractor):
    """
    Architecture CNN personnalisée pour la grille Snake 30x30.
    
    Plus légère que NatureCNN (conçu pour Atari 84x84),
    adaptée à notre taille d'image plus petite.
    """
    def __init__(self, observation_space, features_dim=256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        
        n_input_channels = observation_space.shape[2]  # 3 canaux
        
        self.cnn = nn.Sequential(
            # Couche 1: 3 -> 32 canaux, kernel 3x3
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 30x30 -> 15x15
            
            # Couche 2: 32 -> 64 canaux
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 15x15 -> 7x7
            
            # Couche 3: 64 -> 64 canaux
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 7x7 -> 3x3
            
            nn.Flatten(),
        )
        
        # Calculer la taille après les convolutions
        # 64 canaux * 3 * 3 = 576
        self.linear = nn.Sequential(
            nn.Linear(64 * 3 * 3, features_dim),
            nn.ReLU(),
        )
    
    def forward(self, observations):
        # SB3 attend (batch, height, width, channels)
        # PyTorch Conv2d attend (batch, channels, height, width)
        x = observations.permute(0, 3, 1, 2).float() / 255.0
        return self.linear(self.cnn(x))

print("Architecture CustomCNN définie.")
print("Structure: Conv(32) -> Pool -> Conv(64) -> Pool -> Conv(64) -> Pool -> FC(256)")

### 6.8 Initialisation du modèle V3

In [ ]:
# Configuration V3
MODELS_DIR_V3 = "checkpoints/PPO_CNN"
TIMESTEPS_V3 = 3_000_000  # 3 millions de pas (CNN converge plus lentement)
SAVE_FREQ_V3 = 200_000    # Sauvegarde tous les 200k pas

os.makedirs(MODELS_DIR_V3, exist_ok=True)

# Création de l'environnement CNN
env_v3 = SnakeEnvCNN()

# Callback pour checkpoints
checkpoint_callback_v3 = CheckpointCallback(
    save_freq=SAVE_FREQ_V3,
    save_path=MODELS_DIR_V3,
    name_prefix="snake_v3_cnn"
)

# Configuration du modèle avec CNN personnalisé
policy_kwargs_v3 = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256),
    net_arch=[128, 64],  # Couches fully-connected après l'extracteur
)

# Initialisation du modèle PPO avec CnnPolicy-like
model_v3 = PPO(
    policy="MlpPolicy",  # On utilise MlpPolicy avec notre extracteur custom
    env=env_v3,
    verbose=1,
    tensorboard_log=LOG_DIR,
    learning_rate=0.0001,      # Plus faible pour stabilité
    gamma=0.99,
    n_steps=2048,              # Plus de steps par update
    batch_size=64,
    n_epochs=10,
    clip_range=0.2,
    policy_kwargs=policy_kwargs_v3
)

print("=" * 60)
print("Modèle V3 (CNN) initialisé")
print("=" * 60)
print(f"Extracteur de features : CustomCNN (256 features)")
print(f"Couches FC après CNN   : [128, 64]")
print(f"Learning rate          : 0.0001")
print(f"Timesteps prévus       : {TIMESTEPS_V3:,}")
print(f"Checkpoints tous les   : {SAVE_FREQ_V3:,} steps")
print("=" * 60)

### 6.9 Lancement de l'entraînement V3

⚠️ **Attention** : L'entraînement de 3 millions de timesteps avec CNN est significativement plus long que les versions MLP (environ 1-2 heures selon votre machine).

In [ ]:
print(f"Démarrage de l'entraînement V3 CNN ({TIMESTEPS_V3:,} timesteps)...")
print(f"Checkpoints sauvegardés dans : {MODELS_DIR_V3}")
print("-" * 60)

# Entraînement avec callback
model_v3.learn(
    total_timesteps=TIMESTEPS_V3,
    callback=checkpoint_callback_v3
)

# Sauvegarde finale
model_v3.save(f"{MODELS_DIR_V3}/snake_v3_cnn_final")

print("-" * 60)
print("Entraînement V3 terminé !")
print(f"Modèle sauvegardé : {MODELS_DIR_V3}/snake_v3_cnn_final.zip")

### 6.10 Évaluation de la V3

In [ ]:
# Évaluation sur 20 épisodes
env_eval_v3 = SnakeEnvCNN()
scores_v3, lengths_v3 = evaluate_model(model_v3, env_eval_v3, n_episodes=20)
env_eval_v3.close()

print("=" * 60)
print("Résultats V3 (CNN) sur 20 épisodes:")
print("=" * 60)
print(f"  Score moyen     : {np.mean(scores_v3):.2f} (+/- {np.std(scores_v3):.2f})")
print(f"  Score max       : {max(scores_v3)}")
print(f"  Score min       : {min(scores_v3)}")
print(f"  Durée moyenne   : {np.mean(lengths_v3):.0f} steps")
print("=" * 60)

### 6.11 Visualisation du jeu V3

Lancez cette cellule pour voir l'agent CNN jouer. Vous devriez observer un comportement plus "anticipatif" : l'agent évite de s'enfermer lui-même.

In [ ]:
def play_game_cnn(model, n_games=3, delay=0.05):
    """
    Lance le jeu avec l'environnement CNN pour visualiser l'agent V3.
    """
    env_play = SnakeEnvCNN(render_mode="human")
    
    for game in range(n_games):
        obs, _ = env_play.reset()
        done = False
        print(f"\nPartie {game + 1}/{n_games}")
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env_play.step(action)
            time.sleep(delay)
        
        print(f"  Score final : {env_play.score}")
    
    env_play.close()
    print("\nVisualisations terminées.")

# Lancer 3 parties avec le modèle V3
print("Lancement du jeu avec le modèle V3 (CNN)...")
play_game_cnn(model_v3, n_games=3, delay=0.05)

---

## 7. Analyse comparative V1 vs V2 vs V3

### 7.1 Comparaison des performances

Cette section compare les trois versions entraînées sur les mêmes métriques.

In [ ]:
# Évaluation comparative sur 50 épisodes pour chaque version
N_EVAL_EPISODES = 50

print("Évaluation comparative sur 50 épisodes par version...")
print("-" * 60)

# V1
env_v1_eval = SnakeEnv()
scores_v1_full, lengths_v1_full = evaluate_model(model_v1, env_v1_eval, n_episodes=N_EVAL_EPISODES)
env_v1_eval.close()
print(f"V1 terminée : score moyen = {np.mean(scores_v1_full):.2f}")

# V2
env_v2_eval = SnakeEnv()
scores_v2_full, lengths_v2_full = evaluate_model(model_v2, env_v2_eval, n_episodes=N_EVAL_EPISODES)
env_v2_eval.close()
print(f"V2 terminée : score moyen = {np.mean(scores_v2_full):.2f}")

# V3
env_v3_eval = SnakeEnvCNN()
scores_v3_full, lengths_v3_full = evaluate_model(model_v3, env_v3_eval, n_episodes=N_EVAL_EPISODES)
env_v3_eval.close()
print(f"V3 terminée : score moyen = {np.mean(scores_v3_full):.2f}")

print("-" * 60)
print("Évaluation terminée.")

### 7.2 Visualisation des résultats

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Graphique 1: Box plot des scores
data_scores = [scores_v1_full, scores_v2_full, scores_v3_full]
bp = axes[0].boxplot(data_scores, labels=['V1 (MLP 64x64)', 'V2 (MLP 128x128)', 'V3 (CNN)'], patch_artist=True)
colors = ['#ff9999', '#99ff99', '#9999ff']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
axes[0].set_ylabel('Score')
axes[0].set_title('Distribution des scores par version')
axes[0].grid(axis='y', alpha=0.3)

# Graphique 2: Barres de score moyen avec écart-type
versions = ['V1', 'V2', 'V3']
means = [np.mean(scores_v1_full), np.mean(scores_v2_full), np.mean(scores_v3_full)]
stds = [np.std(scores_v1_full), np.std(scores_v2_full), np.std(scores_v3_full)]
bars = axes[1].bar(versions, means, yerr=stds, capsize=5, color=colors, edgecolor='black')
axes[1].set_ylabel('Score moyen')
axes[1].set_title('Score moyen (+/- écart-type)')
axes[1].grid(axis='y', alpha=0.3)

# Ajouter les valeurs sur les barres
for bar, mean, std in zip(bars, means, stds):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.5,
                f'{mean:.1f}', ha='center', va='bottom', fontsize=10)

# Graphique 3: Score max et durée moyenne
x = np.arange(3)
width = 0.35
max_scores = [max(scores_v1_full), max(scores_v2_full), max(scores_v3_full)]
avg_lengths = [np.mean(lengths_v1_full)/100, np.mean(lengths_v2_full)/100, np.mean(lengths_v3_full)/100]

ax3_twin = axes[2].twinx()
bars1 = axes[2].bar(x - width/2, max_scores, width, label='Score max', color='coral')
bars2 = ax3_twin.bar(x + width/2, avg_lengths, width, label='Durée moy. (x100)', color='skyblue')

axes[2].set_ylabel('Score maximum', color='coral')
ax3_twin.set_ylabel('Durée moyenne (x100 steps)', color='skyblue')
axes[2].set_xticks(x)
axes[2].set_xticklabels(versions)
axes[2].set_title('Score max et durée moyenne')
axes[2].legend(loc='upper left')
ax3_twin.legend(loc='upper right')

plt.suptitle('Comparaison des performances V1 vs V2 vs V3', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 7.3 Tableau récapitulatif

In [ ]:
import pandas as pd

# Création du tableau récapitulatif
summary_data = {
    'Version': ['V1 (Baseline)', 'V2 (MLP amélioré)', 'V3 (CNN)'],
    'Architecture': ['MLP [64, 64]', 'MLP [128, 128]', 'CNN + [128, 64]'],
    'Observation': ['Vecteur (11)', 'Vecteur (11)', 'Image (30x30x3)'],
    'Timesteps': ['100K', '2M', '3M'],
    'Score moyen': [f"{np.mean(scores_v1_full):.2f} ± {np.std(scores_v1_full):.2f}",
                   f"{np.mean(scores_v2_full):.2f} ± {np.std(scores_v2_full):.2f}",
                   f"{np.mean(scores_v3_full):.2f} ± {np.std(scores_v3_full):.2f}"],
    'Score max': [max(scores_v1_full), max(scores_v2_full), max(scores_v3_full)],
    'Durée moy.': [f"{np.mean(lengths_v1_full):.0f}", 
                  f"{np.mean(lengths_v2_full):.0f}",
                  f"{np.mean(lengths_v3_full):.0f}"]
}

df_summary = pd.DataFrame(summary_data)
print("=" * 80)
print("TABLEAU RÉCAPITULATIF DES PERFORMANCES")
print("=" * 80)
print(df_summary.to_string(index=False))
print("=" * 80)

---

## 8. Conclusion

### 8.1 Bilan des expérimentations

Ce projet a permis d'explorer l'impact de différents choix de conception sur les performances d'un agent RL pour le jeu Snake :

1. **V1 (Baseline)** : Configuration minimale avec observation vectorielle et réseau MLP simple. Permet d'établir une référence de performance.

2. **V2 (MLP amélioré)** : Augmentation de la capacité du réseau et du temps d'entraînement. Amélioration des performances mais limitation fondamentale due à l'observation locale.

3. **V3 (CNN)** : Passage à une représentation spatiale complète. Le réseau convolutif peut détecter les patterns spatiaux (impasses, forme du corps) et anticiper les situations dangereuses.

### 8.2 Observations clés

- **La représentation compte plus que la capacité du réseau** : Même un MLP plus large (V2) ne peut pas surmonter les limitations d'une observation "myope".

- **Les CNN sont adaptés aux problèmes spatiaux** : La capacité à détecter des patterns dans une image permet une prise de décision plus "intelligente".

- **Le temps d'entraînement augmente avec la complexité** : Les CNN nécessitent significativement plus de timesteps pour converger.

### 8.3 Pistes d'amélioration futures

- **Reward shaping** : Ajouter des récompenses intermédiaires (se rapprocher de la pomme, maintenir de l'espace libre)
- **Frame stacking** : Empiler plusieurs frames pour donner une notion de mouvement/direction
- **Curriculum learning** : Commencer sur une petite grille puis augmenter progressivement
- **Autres algorithmes** : Tester DQN, A2C, ou SAC pour comparer
- **Self-play** : Entraîner plusieurs agents en compétition